# What is the objective?
Given a large dataframe with the following simple structure:

id|text|create|
--|--|--|
100|hello|1-Jan
200|world|2-Jan

Study the following behaviour:
- How long does it take to load the frame from picked file and search for records with specified ids
- How long does it take to load the frame from picked file , add N new records, delete N oldest records and then save to disk

# What did we learn with 1000,000 rows ?
|Action performed| Time taken(s)|
|--|--|
|Saving the frame using `to_pickle()` (53MB file)| < 1.0|
|Loading the frame using `read_pickle()` | < 1.0|
|Filtering for a single record  | 0.3|
|Searching for 5000 records using `isin()` | 0.2 |
|Adding 100 new records to 1 million records using `concat()` | < 0.3 |



# Create dataframe

In [19]:
import random
import pandas as pd
import time
import datetime as dt


# Given list of N integers
n_integers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]



def pick_randomly_from_list(input: list[int] ,m_times: int)->list[int]:
    n_integers = input
    random_picks = [random.choice(n_integers) for _ in range(m_times)]
    return random_picks

In [12]:

# Given list of N integers
n_integers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Number of times to pick randomly
m_times = 50
results=pick_randomly_from_list(input=n_integers, m_times=m_times)
print("Random Picks:", results)


Random Picks: [5, 6, 8, 4, 10, 7, 9, 3, 5, 7, 1, 4, 4, 7, 3, 6, 2, 1, 2, 1, 7, 1, 2, 10, 10, 4, 10, 4, 6, 5, 6, 4, 6, 8, 9, 4, 7, 10, 9, 10, 5, 2, 10, 10, 10, 3, 8, 7, 5, 9]


In [ ]:
def create_dataframe(max_rows: int, unique_row_count: int)->pd.DataFrame:
    starting_number=10000
    gap_between_record_minutes=3
    unique_list = list(range(starting_number, starting_number + unique_row_count))
    random_picks = pick_randomly_from_list(input=unique_list, m_times=max_rows)
    random_strings = [f"This is string number: {num}" for num in random_picks]
    create_dates: list[dt.datetime] = []
    now = dt.datetime.now()
    for idx in range(max_rows):
        few_minutes_ago = now - dt.timedelta(minutes=idx*gap_between_record_minutes)
        create_dates.append(few_minutes_ago)

    df = pd.DataFrame({
        'id': random_picks,
        'description': random_strings,
        'created_at': create_dates
    })
    return df

# Demo with 100,000 records

In [ ]:
# Save the dataframe to a binary format
binary_file_path = 'sample_dataframe.pkl'
df.to_pickle(binary_file_path)

print(f"Dataframe saved to {binary_file_path}")

          id                   description                 created_at
0      10117  This is string number: 10117 2025-11-11 22:05:04.178859
1      10154  This is string number: 10154 2025-11-11 22:02:04.178859
2      10424  This is string number: 10424 2025-11-11 21:59:04.178859
3      10325  This is string number: 10325 2025-11-11 21:56:04.178859
4      10590  This is string number: 10590 2025-11-11 21:53:04.178859
...      ...                           ...                        ...
99995  10101  This is string number: 10101 2025-04-17 14:20:04.178859
99996  10217  This is string number: 10217 2025-04-17 14:17:04.178859
99997  10765  This is string number: 10765 2025-04-17 14:14:04.178859
99998  10496  This is string number: 10496 2025-04-17 14:11:04.178859
99999  10584  This is string number: 10584 2025-04-17 14:08:04.178859

[100000 rows x 3 columns]


# Save 100,000 to disk

In [ ]:

# Save the dataframe to a pickle file and measure the time taken
binary_file_path = 'sample_dataframe.pkl'
start_time = time.time()
df_sample_100_000.to_pickle(binary_file_path)
end_time = time.time()

time_taken = end_time - start_time
print(f"Dataframe saved to {binary_file_path} in {time_taken:.4f} seconds")

Dataframe saved to sample_dataframe.pkl in 0.0630 seconds


# 1000,000 records
## Create the frame

In [67]:
start_time = time.time()
df_1000_000=create_dataframe(max_rows=1000000, unique_row_count=10000)
end_time = time.time()
time_taken = end_time - start_time
print(f"Dataframe created in {time_taken:.4f} seconds")

Dataframe created in 22.9484 seconds


## Save to disk

In [68]:
binary_file_path = 'sample_dataframe.pkl'

start_time = time.time()
df_1000_000.to_pickle(binary_file_path)
end_time = time.time()

time_taken = end_time - start_time
print(f"Dataframe saved to {binary_file_path} in {time_taken:.4f} seconds")


Dataframe saved to sample_dataframe.pkl in 0.3339 seconds


## Load from disk

In [83]:
binary_file_path = 'sample_dataframe.pkl'
start_time = time.time()
df_loaded = pd.read_pickle(binary_file_path)
end_time = time.time()
time_taken = end_time - start_time
print(f"Dataframe loaded from {binary_file_path} in {time_taken:.4f} seconds")
print(f"Loaded Dataframe shape: {df_loaded.shape}")

Dataframe loaded from sample_dataframe.pkl in 0.2911 seconds
Loaded Dataframe shape: (1000100, 3)


In [40]:
df_loaded

,id,description,created_at
0,14955,This is string number: 14955,2025-11-11 22:21:57.488869
1,16291,This is string number: 16291,2025-11-11 22:18:57.488869
2,16247,This is string number: 16247,2025-11-11 22:15:57.488869
3,19501,This is string number: 19501,2025-11-11 22:12:57.488869
4,12050,This is string number: 12050,2025-11-11 22:09:57.488869
...,...,...,...
999995,16014,This is string number: 16014,2020-02-28 14:36:57.488869
999996,16498,This is string number: 16498,2020-02-28 14:33:57.488869
999997,19564,This is string number: 19564,2020-02-28 14:30:57.488869
999998,18631,This is string number: 18631,2020-02-28 14:27:57.488869


## Search for single records

In [70]:
unique_ids = df_loaded["id"].unique().tolist()
print(f"Unique IDs in the dataframe: {len(unique_ids)}")

random_id = random.choice(unique_ids)
print(f"Filtering dataframe for id: {random_id}")
filtered_df = df_loaded[df_loaded["id"] == random_id]
print(f"Filtered Dataframe shape: {filtered_df.shape}")

Unique IDs in the dataframe: 10000
Filtering dataframe for id: 10276
Filtered Dataframe shape: (94, 3)


## Search multiple

In [73]:
unique_ids = df_loaded["id"].unique().tolist()
print(f"Unique IDs in the dataframe: {len(unique_ids)}")

k=int(len(unique_ids)/2)
random_ids = random.sample(unique_ids, k=k)
random_ids.sort()
print(f"Filtering dataframe for ids: {len(random_ids)}")
start_time = time.time()
filtered_df = df_loaded[df_loaded["id"].isin(random_ids)]

print(f"Filtered Dataframe shape: {filtered_df.shape}")
unique_ids_after_filtering = filtered_df["id"].unique().tolist()
unique_ids_after_filtering.sort()
print(f"Unique IDs after filtering: {unique_ids_after_filtering}")

end_time = time.time()
time_taken = end_time - start_time
print(f"Dataframe search for {len(random_ids)} ids in {time_taken:.4f} seconds")



Unique IDs in the dataframe: 10000
Filtering dataframe for ids: 5000
Filtered Dataframe shape: (499736, 3)
Unique IDs after filtering: [10002, 10004, 10005, 10007, 10012, 10013, 10016, 10018, 10021, 10024, 10025, 10026, 10027, 10031, 10034, 10036, 10038, 10040, 10042, 10046, 10048, 10051, 10052, 10055, 10060, 10062, 10063, 10064, 10066, 10068, 10069, 10070, 10071, 10074, 10075, 10080, 10081, 10082, 10084, 10092, 10093, 10099, 10102, 10104, 10107, 10109, 10112, 10115, 10116, 10118, 10119, 10121, 10122, 10123, 10124, 10125, 10126, 10128, 10130, 10131, 10132, 10133, 10135, 10138, 10140, 10141, 10144, 10152, 10154, 10157, 10159, 10161, 10162, 10163, 10167, 10169, 10170, 10171, 10173, 10174, 10177, 10181, 10183, 10184, 10187, 10189, 10190, 10191, 10192, 10194, 10195, 10200, 10202, 10203, 10204, 10208, 10211, 10213, 10215, 10217, 10220, 10221, 10222, 10223, 10227, 10228, 10231, 10232, 10233, 10234, 10236, 10238, 10240, 10244, 10246, 10247, 10248, 10249, 10251, 10252, 10256, 10257, 10258, 102

## Append records

In [74]:
df_100=create_dataframe(max_rows=100, unique_row_count=10)
print(f"Generated dataframe with shape: {df_100.shape}")

Generated dataframe with shape: (100, 3)


In [80]:
df_concat = pd.concat([df_1000_000,df_100])
print(f"Combined dataframe shape: {df_concat.shape}")
df_concat.reindex()
df_concat

Combined dataframe shape: (1000100, 3)


,id,description,created_at
0,17631,This is string number: 17631,2025-11-13 07:46:37.415697
1,15130,This is string number: 15130,2025-11-13 07:43:37.415697
2,17604,This is string number: 17604,2025-11-13 07:40:37.415697
3,16029,This is string number: 16029,2025-11-13 07:37:37.415697
4,11575,This is string number: 11575,2025-11-13 07:34:37.415697
...,...,...,...
95,10000,This is string number: 10000,2025-11-13 03:08:12.089753
96,10001,This is string number: 10001,2025-11-13 03:05:12.089753
97,10005,This is string number: 10005,2025-11-13 03:02:12.089753
98,10004,This is string number: 10004,2025-11-13 02:59:12.089753


In [82]:
binary_file_path = 'sample_dataframe.pkl'

start_time = time.time()
df_concat.to_pickle(binary_file_path)
end_time = time.time()

time_taken = end_time - start_time
print(f"Dataframe saved to {binary_file_path} in {time_taken:.4f} seconds")

Dataframe saved to sample_dataframe.pkl in 0.7973 seconds
